In [1]:
from find_prices_nonoverlap import *
from sklearn import metrics
from sklearn.model_selection import (GridSearchCV, StratifiedKFold, cross_val_score)
import warnings
warnings.filterwarnings(action='once')

In [2]:
df = pd.read_csv('single_family_home_values.csv')

print('Preparing Data: cleaning data')
not_useful_fields = ['id','city','state']#, 'address']
date_fields = ['lastSaleDate', 'priorSaleDate']
required_fields = ['address','latitude','longitude','zipcode','bedrooms',
                   'bathrooms','rooms','squareFootage',
                   'lotSize','yearBuilt', 'lastSaleAmount' 
                   'lastSaleDate','estimated_value']
zero_to_nan_fields = None
nan_to_zero_fields = ['latitude','longitude','priorSaleDate','priorSaleAmount']
# categorical_fields = ['zipcode']
# remove_outliers=False
# df2, new_fields = prepare_data(df, not_useful_fields=not_useful_fields,
#         date_fields=date_fields,
#         remove_outliers=remove_outliers,
#         required_fields=required_fields,
#         zero_to_nan_fields=zero_to_nan_fields,
#         nan_to_zero_fields=nan_to_zero_fields,
#         )
#         categorical_fields=categorical_fields,
print('Preparing Data: splitting test/train/validation')
df_train, df_test, df_validation = train_validate_test_split(df)

X_test = df_test[df_test.columns.drop('estimated_value')]
y_test = df_test['estimated_value']

X_train = df_train[df_train.columns.drop('estimated_value')]
y_train = df_train['estimated_value'] 

Preparing Data: cleaning data
Preparing Data: splitting test/train/validation


In [3]:
#tzip = ImputeZipCodes()
#tzip.fit_transform(X_train,y_train)

In [ ]:
from sklearn.ensemble import (
    ExtraTreesRegressor, GradientBoostingRegressor, AdaBoostRegressor 
)

print("Setting up pipeline")
features = ('latitude', 'longitude', 'zipcode',
            'bedrooms', 'bathrooms','rooms', 
            'squareFootage', 'lotSize', 
            'yearBuilt',
            'lastSaleAmount','lastSaleDateYear','lastSaleDateMonth','lastSaleDateDayOfWeek')


#: Note imputer will strip away column heads, it has to be after featurize
#reg_function = LGBMRegressor()
#reg_function = XGBRegressor()
reg_function = RandomForestRegressor(n_jobs=3)
#reg_function = GradientBoostingRegressor()
#reg_function = ExtraTreesRegressor(n_estimators=2)
#reg_function = AdaBoostRegressor(); #Really bad

# prepare_data_pipeline = Pipeline([
#             ('impute_zip_codes', ImputeZipCodes()),
#           ('featurize', featurize(features)),
#           ('imputer', Imputer(missing_values=np.nan, strategy="mean", axis=0)),

#             ])

prepare_data_pipeline = Pipeline([
            ('prepare_data', PrepareData(required_fields=required_fields,
                                         date_fields=date_fields,
                                        zero_to_nan_fields=zero_to_nan_fields,
                                        nan_to_zero_fields=nan_to_zero_fields)), 
          ('featurize', featurize(features)),
          ('imputer', Imputer(missing_values=np.nan, strategy="mean", axis=0)),
            ])

fit_data_pipeline = Pipeline([
          ('forest', reg_function)
            ])

Setting up pipeline


In [ ]:
x_train = prepare_data_pipeline.fit_transform(X = df_train, y = y_train)

print("Fitting model on training data")
model = fit_data_pipeline.fit(X = x_train, y = y_train)

print("Evaluating model on training data")
y_pred = model.predict(x_train)
print_metrics(y_train, y_pred)
plt.plot((y_train), (y_pred),'.')
y_max = (np.max((y_train, y_pred)))
plt.plot((0,y_max), (0,y_max),color='black')
plt.show()

print("Evaluating model on testing data")
X_test = prepare_data_pipeline.transform(df_test)
y_test_pred = model.predict(X_test)
print_metrics(y_test, y_test_pred)
plt.plot((y_test), (y_test_pred),'.', alpha=0.2)
y_max = (np.max((y_test, y_test_pred)))
plt.plot((0,y_max), (0,y_max),color='black')
plt.xlim(0,1e6)
plt.ylim(0,1e6)
plt.show()

/Users/derringi/.edm/envs/may2018/lib/python2.7/site-packages/IPython/core/debugger.py:243: DeprecationWarning: The `color_scheme` argument is deprecated since version 5.1
  DeprecationWarning)


> /Users/derringi/Dropbox/projects/job_projects/housing_prices/find_prices_nonoverlap.py(121)transform()
    120         import ipdb; ipdb.set_trace()
--> 121         df = self.keep_required_fields.transform(df)
    122         df = self.zero_to_nan.transform(df)

ipdb> c


find_prices_nonoverlap.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df.drop(labels=key, axis=1, inplace=True)


> /Users/derringi/Dropbox/projects/job_projects/housing_prices/find_prices_nonoverlap.py(96)transform()
     95         import ipdb; ipdb.set_trace()
---> 96         return df
     97 

ipdb> df.keys()
Index([u'address', u'zipcode', u'latitude', u'longitude', u'bedrooms',
       u'bathrooms', u'rooms', u'squareFootage', u'lotSize', u'yearBuilt',
       u'estimated_value'],
      dtype='object')


In [ ]:
 ['address','latitude','longitude','zipcode','bedrooms',
                   'bathrooms','rooms','squareFootage',
                   'lotSize','yearBuilt', 'lastSaleAmount' 
                   'lastSaleDate','estimated_value']

In [ ]:
[c,b] = np.histogram(y_pred-y, bins=np.linspace(-1e6,1e6,100))
print(np.mean(y_pred-y))
plt.plot(b[:-1],c)
plt.show()

In [ ]:

sns.distplot(np.log(df['lastSaleAmount'].values+1))
 plt.show()


In [ ]:
#: Work on using grid-searches
params = {'forest__n_estimators':  (39,42)}
# params = {'forest__n_estimators':  np.arange(3,31,3)}
from sklearn.preprocessing import FunctionTransformer
# transformer = FunctionTransformer(np.log1p)
# transformer.transform(y)
grid = GridSearchCV(pipeline, params)
grid.fit(X = X, y = y)

#: Definining a specified cross-validation technique
cv= StratifiedKFold(n_splits=2)

# Alternatively, can say can say cv=3 to say a 3-repeat cross-validator
grid_search = GridSearchCV(pipeline, params)#, cv=cv)
#gsearch = GridSearchCV(pipeline, params)
grid_search.fit(X = X, y = y)
best_pipeline = grid_search.best_estimator_
best_pipeline.get_params()['forest']

In [ ]:
best_pipeline = grid_search.best_estimator_
best_pipeline.get_params()['forest']

In [ ]:
X = X_test
y=y_test
y_pred = best_pipeline.predict(X)
print(np.sqrt(metrics.mean_squared_error(y,y_pred)))
print(metrics.mean_absolute_error(y,y_pred))
print(metrics.r2_score(y,y_pred))
print(abs_mean_relative_error(y,y_pred))
plt.plot(y, y_pred,'.')
y_max = np.max((y, y_pred))
plt.plot((0,y_max), (0,y_max),color='black')
plt.show()

In [ ]:
required_fields = ['address','latitude','longitude','zipcode','bedrooms',
                   'bathrooms','rooms','squareFootage',
                   'lotSize','yearBuilt',
                   'lastSaleDate','estimated_value']
for key in iter(df.keys()):
    if key not in required_fields:
        print(key)

In [ ]:
df.drop(column=['id'])

In [ ]:
df.drop(labels='id',axis=1)

In [ ]:
df